In [1]:
import os
import random
from math import sqrt
import json
import numpy as np
import pandas as pd
import lightgbm as lgb
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.metrics import cohen_kappa_score, make_scorer, mean_squared_error
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [2]:
def _log(str):
    os.system(f'echo \"{str}\"')
    print(str)

In [3]:
NAN = '__NAN__'
INPUT_ROOT = '../input/data-science-bowl-2019'
JOIN_KEY = ['installation_id', 'game_session', 'title']
TARGET = 'accuracy_group'
FEATURES = {
    'event_id', 
    'game_session', 
    'timestamp', 
    'installation_id', 
    'event_count',
    'event_code', 
    'game_time', 
    'title', 
    'type', 
    'world',
    'event_data'
}
EVENT_CODES = ['2000', '2010', '2020', '2025', '2030', '2035', '2040', '2050', '2060', '2070', '2075', '2080', '2081', '2083', '3010', '3020', '3021', '3110', '3120', '3121', '4010', '4020', '4021', '4022', '4025', '4030', '4031', '4035', '4040', '4045', '4050', '4070', '4080', '4090', '4095', '4100', '4110', '4220', '4230', '4235', '5000', '5010']
SEED = 31
FOLDS = 3
ESTIMATORS = 1000

In [4]:
def _init():
    # Characters such as empty strings '' or numpy.inf are considered NA values
    pd.set_option('use_inf_as_na', True)
    pd.set_option('display.max_columns', 999)
    pd.set_option('display.max_rows', 999)
    
    
_init()

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)


seed_everything(SEED)

In [6]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk(INPUT_ROOT):
    for filename in filenames:
        _log(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

../input/data-science-bowl-2019\sample_submission.csv
../input/data-science-bowl-2019\test.csv
../input/data-science-bowl-2019\test.csv.zip
../input/data-science-bowl-2019\train.csv
../input/data-science-bowl-2019\train.csv.zip
../input/data-science-bowl-2019\train_labels.csv
../input/data-science-bowl-2019\train_labels.csv.zip


In [7]:
%%time
train_raw = pd.read_csv(f'{INPUT_ROOT}/train.csv', usecols=FEATURES)
train_labels = pd.read_csv(f'{INPUT_ROOT}/train_labels.csv', usecols=JOIN_KEY + [TARGET])
test_raw = pd.read_csv(f'{INPUT_ROOT}/test.csv', usecols=FEATURES)
train_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17690 entries, 0 to 17689
Data columns (total 4 columns):
game_session       17690 non-null object
installation_id    17690 non-null object
title              17690 non-null object
accuracy_group     17690 non-null int64
dtypes: int64(1), object(3)
memory usage: 552.9+ KB
Wall time: 53.9 s


# Add labels to train data

In [8]:
def _remove_unlabelled_data(train_raw, train_labels):
    return train_raw[train_raw['installation_id'].isin(train_labels['installation_id'].unique())]


train_raw = _remove_unlabelled_data(train_raw, train_labels)

In [9]:
%%time
def _add_labels(train_raw, train_labels, on):
    train_raw = pd.merge(train_raw, train_labels, on=on, how='left')
    train_raw[TARGET] = np.int16(train_raw[TARGET])
    return train_raw


train_raw = _add_labels(train_raw, train_labels, on=JOIN_KEY)
del train_labels

Wall time: 4.55 s


# Extract event data JSON

In [10]:
def _concat_columns(df1, df2):
    """Concatenate the columns of two pandas dataframes in the order of the operands.
    Both dataframes must have the same number of rows.
    """
    assert len(df1) == len(df2)
    res = pd.concat([df1, df2.reindex(df1.index)], axis=1, join='inner')
    assert len(res) == len(df1)
    return res
    

def _extract_event_data(df, keep_cols, chunk_size=100000):
    res = pd.DataFrame()
    _len = len(df)
    all_cols = []
    for i in tqdm(range(0, _len, chunk_size)):
        if i + chunk_size < _len:
            chunk = df[i:i + chunk_size].copy()
        else:
            chunk = df[i:].copy()
        ed = pd.io.json.json_normalize(chunk['event_data'].apply(json.loads)).add_prefix('ed.')
        if i == 0:
            all_cols = ed.columns.values
        ed = ed[keep_cols]
        chunk = _concat_columns(chunk, ed)
        res = pd.concat([res, chunk], ignore_index=True)
    _log(f'{len(all_cols)} event_data={all_cols}')
    assert len(df) == len(res)
    return res


keep_cols = ['ed.identifier', 'ed.duration', 'ed.coordinates.x', 'ed.coordinates.y',
            'ed.coordinates.stage_width', 'ed.coordinates.stage_height',
            'ed.level', 'ed.round', 'ed.correct', 'ed.misses']
train_raw = _extract_event_data(train_raw, keep_cols)
test_raw = _extract_event_data(test_raw, keep_cols)

  0%|                                                           | 0/12 [00:00<?, ?it/s]

136 event_data=['ed.event_code' 'ed.event_count' 'ed.version' 'ed.game_time'
 'ed.description' 'ed.identifier' 'ed.media_type' 'ed.total_duration'
 'ed.coordinates.x' 'ed.coordinates.y' 'ed.coordinates.stage_width'
 'ed.coordinates.stage_height' 'ed.duration' 'ed.size' 'ed.sand'
 'ed.filled' 'ed.castles_placed' 'ed.molds' 'ed.level' 'ed.round'
 'ed.movie_id' 'ed.time_played' 'ed.options' 'ed.animals'
 'ed.round_target.size' 'ed.round_target.type' 'ed.round_target.animal'
 'ed.item_type' 'ed.position' 'ed.animal' 'ed.correct' 'ed.misses'
 'ed.houses' 'ed.dinosaurs' 'ed.dinosaur' 'ed.dinosaurs_placed'
 'ed.house.size' 'ed.house.position' 'ed.rocket' 'ed.height' 'ed.launched'
 'ed.flowers' 'ed.flower' 'ed.growth' 'ed.stumps' 'ed.source'
 'ed.destination' 'ed.session_duration' 'ed.exit_type' 'ed.distance'
 'ed.target_distances' 'ed.round_prompt' 'ed.target_size' 'ed.resources'
 'ed.object_type' 'ed.group' 'ed.bug' 'ed.buglength' 'ed.stage_number'
 'ed.hat' 'ed.caterpillar' 'ed.hats' 'ed.ca

100%|██████████████████████████████████████████████████| 12/12 [01:23<00:00,  6.95s/it]

134 event_data=['ed.event_code' 'ed.event_count' 'ed.version' 'ed.round' 'ed.game_time'
 'ed.coordinates.x' 'ed.coordinates.y' 'ed.coordinates.stage_width'
 'ed.coordinates.stage_height' 'ed.description' 'ed.identifier'
 'ed.media_type' 'ed.total_duration' 'ed.duration' 'ed.dinosaur' 'ed.diet'
 'ed.target_weight' 'ed.resources' 'ed.weight' 'ed.source' 'ed.correct'
 'ed.scale_weight' 'ed.scale_contents' 'ed.houses' 'ed.dinosaurs'
 'ed.size' 'ed.dinosaurs_placed' 'ed.house.size' 'ed.house.position'
 'ed.misses' 'ed.rocket' 'ed.height' 'ed.launched' 'ed.crystals'
 'ed.crystal_id' 'ed.side' 'ed.left' 'ed.right' 'ed.session_duration'
 'ed.layout.row1' 'ed.layout.row2' 'ed.gate.row' 'ed.gate.column'
 'ed.gate.side' 'ed.nest' 'ed.bug' 'ed.sand' 'ed.filled'
 'ed.castles_placed' 'ed.target_water_level' 'ed.water_level' 'ed.level'
 'ed.movie_id' 'ed.time_played' 'ed.options' 'ed.animals'
 'ed.round_target.size' 'ed.round_target.type' 'ed.round_target.animal'
 'ed.item_type' 'ed.position' 'ed.ani

In [11]:
test_raw.info(max_cols=999)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1156414 entries, 0 to 1156413
Data columns (total 21 columns):
event_id                       1156414 non-null object
game_session                   1156414 non-null object
timestamp                      1156414 non-null object
event_data                     1156414 non-null object
installation_id                1156414 non-null object
event_count                    1156414 non-null int64
event_code                     1156414 non-null int64
game_time                      1156414 non-null int64
title                          1156414 non-null object
type                           1156414 non-null object
world                          1156414 non-null object
ed.identifier                  33835 non-null object
ed.duration                    32850 non-null float64
ed.coordinates.x               51261 non-null float64
ed.coordinates.y               51261 non-null float64
ed.coordinates.stage_width     51261 non-null float64
ed.coordinates.s

In [12]:
train_raw.info(max_cols=999)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7734558 entries, 0 to 7734557
Data columns (total 22 columns):
event_id                       object
game_session                   object
timestamp                      object
event_data                     object
installation_id                object
event_count                    int64
event_code                     int64
game_time                      int64
title                          object
type                           object
world                          object
accuracy_group                 int16
ed.identifier                  object
ed.duration                    float64
ed.coordinates.x               float64
ed.coordinates.y               float64
ed.coordinates.stage_width     float64
ed.coordinates.stage_height    float64
ed.level                       float64
ed.round                       float64
ed.correct                     object
ed.misses                      float64
dtypes: float64(8), int16(1), int64(3), object(

In [13]:
# All event ids in test set also exist in train set
test_set = set(test_raw['event_id'])
train_set = set(train_raw['event_id'])
vs = test_set - train_set
_log(f'{len(vs)} event_ids exist in test set but not train set.')

0 event_ids exist in test set but not train set.


In [14]:
EVENT_IDS = sorted(test_raw['event_id'].unique())
_log(f'{len(vs)} EVENT_IDS={vs}')

0 EVENT_IDS=set()


In [15]:
vs = sorted(train_raw['type'].unique())
_log(f'{len(vs)} train_raw type={vs}')

4 train_raw type=['Activity', 'Assessment', 'Clip', 'Game']


In [16]:
vs = sorted(train_raw['world'].unique())
_log(f'{len(vs)} train_raw type={vs}')

4 train_raw type=['CRYSTALCAVES', 'MAGMAPEAK', 'NONE', 'TREETOPCITY']


In [17]:
vs = sorted(train_raw['event_code'].unique())
_log(f'{len(vs)} train_raw type={vs}')

42 train_raw type=[2000, 2010, 2020, 2025, 2030, 2035, 2040, 2050, 2060, 2070, 2075, 2080, 2081, 2083, 3010, 3020, 3021, 3110, 3120, 3121, 4010, 4020, 4021, 4022, 4025, 4030, 4031, 4035, 4040, 4045, 4050, 4070, 4080, 4090, 4095, 4100, 4110, 4220, 4230, 4235, 5000, 5010]


In [18]:
vs = sorted(train_raw['title'].unique())
_log(f'{len(vs)} train_raw titles={vs}')

44 train_raw titles=['12 Monkeys', 'Air Show', 'All Star Sorting', 'Balancing Act', 'Bird Measurer (Assessment)', 'Bottle Filler (Activity)', 'Bubble Bath', 'Bug Measurer (Activity)', 'Cart Balancer (Assessment)', 'Cauldron Filler (Assessment)', 'Chest Sorter (Assessment)', 'Chicken Balancer (Activity)', 'Chow Time', 'Costume Box', 'Crystal Caves - Level 1', 'Crystal Caves - Level 2', 'Crystal Caves - Level 3', 'Crystals Rule', 'Dino Dive', 'Dino Drink', 'Egg Dropper (Activity)', 'Fireworks (Activity)', 'Flower Waterer (Activity)', 'Happy Camel', 'Heavy, Heavier, Heaviest', 'Honey Cake', 'Leaf Leader', 'Lifting Heavy Things', 'Magma Peak - Level 1', 'Magma Peak - Level 2', 'Mushroom Sorter (Assessment)', 'Ordering Spheres', 'Pan Balance', "Pirate's Tale", 'Rulers', 'Sandcastle Builder (Activity)', 'Scrub-A-Dub', 'Slop Problem', 'Treasure Map', 'Tree Top City - Level 1', 'Tree Top City - Level 2', 'Tree Top City - Level 3', 'Watering Hole (Activity)', 'Welcome to Lost Lagoon!']


In [19]:
vs = sorted(test_raw['title'].unique())
_log(f'{len(vs)} test titles={vs}')

44 test titles=['12 Monkeys', 'Air Show', 'All Star Sorting', 'Balancing Act', 'Bird Measurer (Assessment)', 'Bottle Filler (Activity)', 'Bubble Bath', 'Bug Measurer (Activity)', 'Cart Balancer (Assessment)', 'Cauldron Filler (Assessment)', 'Chest Sorter (Assessment)', 'Chicken Balancer (Activity)', 'Chow Time', 'Costume Box', 'Crystal Caves - Level 1', 'Crystal Caves - Level 2', 'Crystal Caves - Level 3', 'Crystals Rule', 'Dino Dive', 'Dino Drink', 'Egg Dropper (Activity)', 'Fireworks (Activity)', 'Flower Waterer (Activity)', 'Happy Camel', 'Heavy, Heavier, Heaviest', 'Honey Cake', 'Leaf Leader', 'Lifting Heavy Things', 'Magma Peak - Level 1', 'Magma Peak - Level 2', 'Mushroom Sorter (Assessment)', 'Ordering Spheres', 'Pan Balance', "Pirate's Tale", 'Rulers', 'Sandcastle Builder (Activity)', 'Scrub-A-Dub', 'Slop Problem', 'Treasure Map', 'Tree Top City - Level 1', 'Tree Top City - Level 2', 'Tree Top City - Level 3', 'Watering Hole (Activity)', 'Welcome to Lost Lagoon!']


In [20]:
def _transform_timestamp(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    return df


train_raw = _transform_timestamp(train_raw)
test_raw = _transform_timestamp(test_raw)

In [21]:
%%time
def _set_string_type(df, cols):
    df[cols] = df[cols].fillna(NAN).astype(str)
    return df


cols = ['event_code', 'timestamp']
train_raw = _set_string_type(train_raw, cols=cols)
test_raw = _set_string_type(test_raw, cols=cols)

Wall time: 1min 21s


In [22]:
%%time
def _sort_it(df):
    return df.sort_values(by=['installation_id', 'timestamp'])


train_raw = _sort_it(train_raw)
test_raw = _sort_it(test_raw)

Wall time: 33.5 s


# Multiple accuracy groups per installation id
In the train set, there are multiple accuracy groups per installation id. The task is to predict the accuracy group of the **last** assessment for a given installation id.

In [23]:
vs = train_raw[train_raw[TARGET].notna()].groupby('installation_id')[TARGET].nunique()
vs

installation_id
0006a69f    3
0006c192    3
00129856    2
001d0ed0    3
00225f67    1
           ..
ff9305d7    2
ff9715db    4
ffc90c32    3
ffd2871d    2
ffeb0b1b    2
Name: accuracy_group, Length: 3614, dtype: int64

In [24]:
train_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7734558 entries, 0 to 7734557
Data columns (total 22 columns):
event_id                       object
game_session                   object
timestamp                      object
event_data                     object
installation_id                object
event_count                    int64
event_code                     object
game_time                      int64
title                          object
type                           object
world                          object
accuracy_group                 int16
ed.identifier                  object
ed.duration                    float64
ed.coordinates.x               float64
ed.coordinates.y               float64
ed.coordinates.stage_width     float64
ed.coordinates.stage_height    float64
ed.level                       float64
ed.round                       float64
ed.correct                     object
ed.misses                      float64
dtypes: float64(8), int16(1), int64(2), object

In [25]:
def _log_smoothing(df, cols):
    for col in cols:
        df[col] = np.log(df[col] + 1)
    return df


#cols = ['event_count', 'game_time']
#train = _log_smoothing(train, cols)
#test = _log_smoothing(test, cols)

In [26]:
train_raw.head(40)

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world,accuracy_group,ed.identifier,ed.duration,ed.coordinates.x,ed.coordinates.y,ed.coordinates.stage_width,ed.coordinates.stage_height,ed.level,ed.round,ed.correct,ed.misses
0,27253bdc,34ba1a28d02ba8ba,2019-08-06 04:57:18.904000+00:00,"{""event_code"": 2000, ""event_count"": 1}",0006a69f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,27253bdc,4b57c9a59474a1b9,2019-08-06 04:57:45.301000+00:00,"{""event_code"": 2000, ""event_count"": 1}",0006a69f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,77261ab5,2b9d5af79bcdb79f,2019-08-06 04:58:14.538000+00:00,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0006a69f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,b2dba42b,2b9d5af79bcdb79f,2019-08-06 04:58:14.615000+00:00,"{""description"":""Let's build a sandcastle! Firs...",0006a69f,2,3010,29,Sandcastle Builder (Activity),Activity,MAGMAPEAK,0,"Dot_LetsSandcastle,Dot_FillMold,Dot_MoldShape",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325467d,2b9d5af79bcdb79f,2019-08-06 04:58:16.680000+00:00,"{""coordinates"":{""x"":273,""y"":650,""stage_width"":...",0006a69f,3,4070,2137,Sandcastle Builder (Activity),Activity,MAGMAPEAK,0,NaN,NaN,273.0,650.0,1015.0,762.0,NaN,NaN,NaN,NaN
5,1325467d,2b9d5af79bcdb79f,2019-08-06 04:58:18.474000+00:00,"{""coordinates"":{""x"":863,""y"":237,""stage_width"":...",0006a69f,4,4070,3937,Sandcastle Builder (Activity),Activity,MAGMAPEAK,0,NaN,NaN,863.0,237.0,1015.0,762.0,NaN,NaN,NaN,NaN
6,1325467d,2b9d5af79bcdb79f,2019-08-06 04:58:19.365000+00:00,"{""coordinates"":{""x"":817,""y"":617,""stage_width"":...",0006a69f,5,4070,4820,Sandcastle Builder (Activity),Activity,MAGMAPEAK,0,NaN,NaN,817.0,617.0,1015.0,762.0,NaN,NaN,NaN,NaN
7,1bb5fbdb,2b9d5af79bcdb79f,2019-08-06 04:58:21.490000+00:00,"{""description"":""Let's build a sandcastle! Firs...",0006a69f,6,3110,6954,Sandcastle Builder (Activity),Activity,MAGMAPEAK,0,"Dot_LetsSandcastle,Dot_FillMold,Dot_MoldShape",6925.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1325467d,2b9d5af79bcdb79f,2019-08-06 04:58:22.732000+00:00,"{""coordinates"":{""x"":809,""y"":180,""stage_width"":...",0006a69f,7,4070,8187,Sandcastle Builder (Activity),Activity,MAGMAPEAK,0,NaN,NaN,809.0,180.0,1015.0,762.0,NaN,NaN,NaN,NaN
9,5e812b27,2b9d5af79bcdb79f,2019-08-06 04:58:23.295000+00:00,"{""size"":0,""coordinates"":{""x"":782,""y"":207,""stag...",0006a69f,8,4030,8745,Sandcastle Builder (Activity),Activity,MAGMAPEAK,0,NaN,NaN,782.0,207.0,1015.0,762.0,NaN,NaN,NaN,NaN


In [27]:
test_raw.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world,ed.identifier,ed.duration,ed.coordinates.x,ed.coordinates.y,ed.coordinates.stage_width,ed.coordinates.stage_height,ed.level,ed.round,ed.correct,ed.misses
0,27253bdc,0ea9ecc81a565215,2019-09-10 16:50:24.910000+00:00,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,27253bdc,c1ea43d8b8261d27,2019-09-10 16:50:55.503000+00:00,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,27253bdc,7ed86c6b72e725e2,2019-09-10 16:51:51.805000+00:00,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,27253bdc,7e516ace50e7fe67,2019-09-10 16:53:12.825000+00:00,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Crystal Caves - Level 1,Clip,CRYSTALCAVES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7d093bf9,a022c3f60ba547e7,2019-09-10 16:54:12.115000+00:00,"{""version"":""1.0"",""round"":0,""event_count"":1,""ga...",00abaee7,1,2000,0,Chow Time,Game,CRYSTALCAVES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN


# Train-test split not by time
Both train and test sets span the same time period.

In [28]:
_log(f'train[timestamp] is from {train_raw.timestamp.min()} to {train_raw.timestamp.max()}')
_log(f'test[timestamp] is from {test_raw.timestamp.min()} to {test_raw.timestamp.max()}')

train[timestamp] is from 2019-07-23 14:38:25.256000+00:00 to 2019-10-14 21:57:26.930000+00:00
test[timestamp] is from 2019-07-24 00:04:25.361000+00:00 to 2019-10-14 21:00:34.858000+00:00


# Feature Engineering

In [33]:
def _cutoff(df, TARGET):
    return df[df[TARGET].notna()]['timestamp'].max()


def _trim_events_after_last_assessment(df, cutoff):
    res = df[df['timestamp'] <= cutoff]
    #_log(f'cutoff: before={df.shape}, after={res.shape}')
    return res
    
    
def _target_variable(df, cutoff, TARGET):
    vs = df[df['timestamp'] == cutoff][TARGET].values
    assert len(set(vs)) == 1
    return int(float(vs[0]))
    
    
def _game_session_stats(df, col):
    vs = df.groupby(['game_session'])[col].transform('max')
    return (vs.median(), vs.max())


def _event_code_counts(df, code):
    total = np.int32([sum(df['event_code'] == code)])
    activity = np.int32([sum((df['event_code'] == code) & (df['type'] == 'Activity'))])
    assessment = np.int32([sum((df['event_code'] == code) & (df['type'] == 'Assessment'))])
    clip = np.int32([sum((df['event_code'] == code) & (df['type'] == 'Clip'))])
    game = np.int32([sum((df['event_code'] == code) & (df['type'] == 'Game'))])
    return (total, activity, assessment, clip, game)


def _event_data_features(df, suffix):
    res = {}
    res[f'ed_duration{suffix}'] = np.int32(df['ed.duration'].fillna(0).max())
    res[f'ed_level{suffix}'] = np.int32(df['ed.level'].fillna(0).max())
    res[f'ed_round{suffix}'] = np.int32(df['ed.round'].fillna(0).max())
    res[f'ed_correct{suffix}'] = np.int32(df['ed.correct'].fillna(0).max())
    res[f'ed_misses{suffix}'] = np.int32(df['ed.misses'].fillna(0).max())
    return res


def _features_map(df, EVENT_CODES, EVENT_IDS, suffix=''):
    res = {}
    res[f'type_activity{suffix}'] = np.int32([sum(df['type'] == 'Activity')])
    res[f'type_assessment{suffix}'] = np.int32([sum(df['type'] == 'Assessment')])
    res[f'type_clip{suffix}'] = np.int32([sum(df['type'] == 'Clip')])
    res[f'type_game{suffix}'] = np.int32([sum(df['type'] == 'Game')])
    assert len(df) == res[f'type_activity{suffix}'][0] + res[f'type_assessment{suffix}'][0] + res[f'type_clip{suffix}'][0] + res[f'type_game{suffix}'][0]
    res[f'world_crystalcaves{suffix}'] = np.int32([sum(df['world'] == 'CRYSTALCAVES')])
    res[f'world_magmapeak{suffix}'] = np.int32([sum(df['world'] == 'MAGMAPEAK')])
    res[f'world_treetopcity{suffix}'] = np.int32([sum(df['world'] == 'TREETOPCITY')])
    res[f'world_none{suffix}'] = np.int32([sum(df['world'] == 'NONE')])
    res[f'title_12_monkeys{suffix}'] = np.int32([sum(df['title'] == '12 Monkeys')])
    res[f'title_air_show{suffix}'] = np.int32([sum(df['title'] == 'Air Show')])
    res[f'title_all_star_sorting{suffix}'] = np.int32([sum(df['title'] == 'All Star Sorting')])
    res[f'title_balancing_act{suffix}'] = np.int32([sum(df['title'] == 'Balancing Act')])
    res[f'title_bird_measurer{suffix}'] = np.int32([sum(df['title'] == 'Bird Measurer (Assessment)')])
    res[f'title_bottle_filler{suffix}'] = np.int32([sum(df['title'] == 'Bottle Filler (Activity)')])
    res[f'title_bubble_bath{suffix}'] = np.int32([sum(df['title'] == 'Bubble Bath')])
    res[f'title_bug_measurer{suffix}'] = np.int32([sum(df['title'] == 'Bug Measurer (Activity)')])
    res[f'title_cart_balancer{suffix}'] = np.int32([sum(df['title'] == 'Cart Balancer (Assessment)')])
    res[f'title_cauldron_filler{suffix}'] = np.int32([sum(df['title'] == 'Cauldron Filler (Assessment)')])
    res[f'title_chest_sorter{suffix}'] = np.int32([sum(df['title'] == 'Chest Sorter (Assessment)')])
    res[f'title_chicken_balancer{suffix}'] = np.int32([sum(df['title'] == 'Chicken Balancer (Activity)')])
    res[f'title_chow_time{suffix}'] = np.int32([sum(df['title'] == 'Chow Time')])
    res[f'title_costume_box{suffix}'] = np.int32([sum(df['title'] == 'Costume Box')])
    res[f'title_crystal_caves_1{suffix}'] = np.int32([sum(df['title'] == 'Crystal Caves - Level 1')])
    res[f'title_crystal_caves_2{suffix}'] = np.int32([sum(df['title'] == 'Crystal Caves - Level 2')])
    res[f'title_crystal_caves_3{suffix}'] = np.int32([sum(df['title'] == 'Crystal Caves - Level 3')])
    res[f'title_crystals_rule{suffix}'] = np.int32([sum(df['title'] == 'Crystals Rule')])
    res[f'title_dino_dive{suffix}'] = np.int32([sum(df['title'] == 'Dino Dive')])
    res[f'title_dino_drink{suffix}'] = np.int32([sum(df['title'] == 'Dino Drink')])
    res[f'title_egg_dropper{suffix}'] = np.int32([sum(df['title'] == 'Egg Dropper (Activity)')])
    res[f'title_fireworks{suffix}'] = np.int32([sum(df['title'] == 'Fireworks (Activity)')])
    res[f'title_flower_waterer{suffix}'] = np.int32([sum(df['title'] == 'Flower Waterer (Activity)')])
    res[f'title_happy_camel{suffix}'] = np.int32([sum(df['title'] == 'Happy Camel')])
    res[f'title_heavy{suffix}'] = np.int32([sum(df['title'] == 'Heavy, Heavier, Heaviest')])
    res[f'title_honey_cake{suffix}'] = np.int32([sum(df['title'] == 'Honey Cake')])
    res[f'title_leaf_leader{suffix}'] = np.int32([sum(df['title'] == 'Leaf Leader')])
    res[f'title_lifting{suffix}'] = np.int32([sum(df['title'] == 'Lifting Heavy Things')])
    res[f'title_magma_peak_1{suffix}'] = np.int32([sum(df['title'] == 'Magma Peak - Level 1')])
    res[f'title_magma_peak_2{suffix}'] = np.int32([sum(df['title'] == 'Magma Peak - Level 2')])
    res[f'title_mushroom_sorter{suffix}'] = np.int32([sum(df['title'] == 'Mushroom Sorter (Assessment)')])
    res[f'title_ordering_spheres{suffix}'] = np.int32([sum(df['title'] == 'Ordering Spheres')])
    res[f'title_pan_balance{suffix}'] = np.int32([sum(df['title'] == 'Pan Balance')])
    res[f'title_pirate_tale{suffix}'] = np.int32([sum(df['title'] == "Pirate's Tale")])
    res[f'title_rulers{suffix}'] = np.int32([sum(df['title'] == 'Rulers')])
    res[f'title_sandcastle{suffix}'] = np.int32([sum(df['title'] == 'Sandcastle Builder (Activity)')])
    res[f'title_scrub{suffix}'] = np.int32([sum(df['title'] == 'Scrub-A-Dub')])
    res[f'title_slop{suffix}'] = np.int32([sum(df['title'] == 'Slop Problem')])
    res[f'title_treasure_map{suffix}'] = np.int32([sum(df['title'] == 'Treasure Map')])
    res[f'title_treetop_city_1{suffix}'] = np.int32([sum(df['title'] == 'Tree Top City - Level 1')])
    res[f'title_treetop_city_2{suffix}'] = np.int32([sum(df['title'] == 'Tree Top City - Level 2')])
    res[f'title_treetop_city_3{suffix}'] = np.int32([sum(df['title'] == 'Tree Top City - Level 3')])
    res[f'title_watering_hole{suffix}'] = np.int32([sum(df['title'] == 'Watering Hole (Activity)')])
    res[f'title_welcome{suffix}'] = np.int32([sum(df['title'] == 'Welcome to Lost Lagoon!')])
    
    for code in EVENT_CODES:
        (total, activity, assessment, clip, game) = _event_code_counts(df, code)
        res[f'event_{code}{suffix}'] = total
        res[f'event_{code}_activity{suffix}'] = activity
        res[f'event_{code}_assessment{suffix}'] = assessment
        res[f'event_{code}_clip{suffix}'] = clip
        res[f'event_{code}_game{suffix}'] = game
    
    for eid in EVENT_IDS:
        res[f'eid_{eid}{suffix}'] = np.int32([sum(df['event_id'] == eid)])
    
    res[f'game_time{suffix}'] = np.int32(df['game_time'].max())
    res[f'event_count{suffix}'] = np.int32(df['event_count'].max())
    res.update(_event_data_features(df, suffix))
    return res


def _features(df, installation_id, EVENT_CODES, EVENT_IDS):
    res = {}
    iid = df[df['installation_id'] == installation_id].copy()
    if TARGET in df.columns:
        cutoff = _cutoff(iid, TARGET)
        iid = _trim_events_after_last_assessment(iid, cutoff)
        res[TARGET] = _target_variable(iid, cutoff, TARGET)
    res['installation_id'] = [installation_id]
    cols = ['game_time', 'event_count']
    for col in cols:
        (_median, _max) = np.int32(_game_session_stats(iid, col))
        res[f'{col}_p50'] = _median
        res[f'{col}_max'] = _max
    res.update(_features_map(iid, EVENT_CODES, EVENT_IDS))
    return pd.DataFrame.from_dict(res)


def _preprocess(raw, EVENT_CODES, EVENT_IDS):
    res = pd.DataFrame()
    iids = raw['installation_id'].unique()
    for iid in tqdm(iids):
        res = pd.concat([res, _features(raw, iid, EVENT_CODES, EVENT_IDS)], ignore_index=True)
    return res


train = _preprocess(train_raw, EVENT_CODES, EVENT_IDS)
train.info(max_cols=999)



  0%|                                                         | 0/3614 [00:00<?, ?it/s]

  0%|                                               | 1/3614 [00:01<1:09:07,  1.15s/it]

  0%|                                               | 2/3614 [00:02<1:06:03,  1.10s/it]

  0%|                                               | 3/3614 [00:02<1:00:54,  1.01s/it]

  0%|                                                 | 4/3614 [00:03<57:51,  1.04it/s]

  0%|                                                 | 5/3614 [00:04<55:19,  1.09it/s]

  0%|                                               | 6/3614 [00:05<1:02:30,  1.04s/it]

  0%|                                               | 7/3614 [00:07<1:03:24,  1.05s/it]

  0%|                                               | 8/3614 [00:07<1:00:52,  1.01s/it]

  0%|                                                 | 9/3614 [00:08<56:08,  1.07it/s]

  0%|▏                                               | 10/3614 [00:09<52:37,  1.14it/s]

  0%|▏             

  2%|█                                             | 86/3614 [01:21<1:04:22,  1.09s/it]

  2%|█                                             | 87/3614 [01:23<1:13:30,  1.25s/it]

  2%|█                                             | 88/3614 [01:24<1:05:09,  1.11s/it]

  2%|█▏                                            | 89/3614 [01:25<1:00:55,  1.04s/it]

  2%|█▏                                              | 90/3614 [01:25<58:01,  1.01it/s]

  3%|█▏                                              | 91/3614 [01:26<54:52,  1.07it/s]

  3%|█▏                                              | 92/3614 [01:27<59:41,  1.02s/it]

  3%|█▏                                              | 93/3614 [01:28<57:10,  1.03it/s]

  3%|█▏                                              | 94/3614 [01:29<54:01,  1.09it/s]

  3%|█▎                                              | 95/3614 [01:30<53:56,  1.09it/s]

  3%|█▎                                              | 96/3614 [01:31<53:05,  1.10it/s]

  3%|█▎              

  5%|██▏                                          | 172/3614 [02:44<1:01:33,  1.07s/it]

  5%|██▏                                            | 173/3614 [02:45<55:29,  1.03it/s]

  5%|██▎                                            | 174/3614 [02:46<53:08,  1.08it/s]

  5%|██▎                                            | 175/3614 [02:47<50:45,  1.13it/s]

  5%|██▎                                            | 176/3614 [02:48<49:36,  1.15it/s]

  5%|██▎                                            | 177/3614 [02:48<47:55,  1.20it/s]

  5%|██▎                                            | 178/3614 [02:49<46:44,  1.23it/s]

  5%|██▎                                            | 179/3614 [02:50<46:07,  1.24it/s]

  5%|██▎                                            | 180/3614 [02:51<47:30,  1.20it/s]

  5%|██▎                                            | 181/3614 [02:52<47:01,  1.22it/s]

  5%|██▎                                            | 182/3614 [02:52<46:02,  1.24it/s]

  5%|██▍             

  7%|███▎                                           | 258/3614 [04:04<50:46,  1.10it/s]

  7%|███▎                                           | 259/3614 [04:05<47:33,  1.18it/s]

  7%|███▍                                           | 260/3614 [04:06<50:35,  1.10it/s]

  7%|███▍                                           | 261/3614 [04:07<49:52,  1.12it/s]

  7%|███▍                                           | 262/3614 [04:08<47:27,  1.18it/s]

  7%|███▍                                           | 263/3614 [04:09<54:24,  1.03it/s]

  7%|███▍                                           | 264/3614 [04:10<57:15,  1.03s/it]

  7%|███▍                                           | 265/3614 [04:11<58:57,  1.06s/it]

  7%|███▍                                           | 266/3614 [04:12<53:16,  1.05it/s]

  7%|███▍                                           | 267/3614 [04:13<51:08,  1.09it/s]

  7%|███▍                                           | 268/3614 [04:14<49:13,  1.13it/s]

  7%|███▍            

 10%|████▍                                          | 344/3614 [05:24<49:54,  1.09it/s]

 10%|████▍                                          | 345/3614 [05:25<49:38,  1.10it/s]

 10%|████▍                                          | 346/3614 [05:26<49:05,  1.11it/s]

 10%|████▌                                          | 347/3614 [05:26<47:28,  1.15it/s]

 10%|████▌                                          | 348/3614 [05:27<49:10,  1.11it/s]

 10%|████▌                                          | 349/3614 [05:28<50:15,  1.08it/s]

 10%|████▌                                          | 350/3614 [05:29<50:17,  1.08it/s]

 10%|████▌                                          | 351/3614 [05:30<49:06,  1.11it/s]

 10%|████▌                                          | 352/3614 [05:31<47:38,  1.14it/s]

 10%|████▌                                          | 353/3614 [05:32<48:40,  1.12it/s]

 10%|████▌                                          | 354/3614 [05:33<47:22,  1.15it/s]

 10%|████▌           

...

 93%|██████████████████████████████████████████▋   | 3354/3614 [54:25<03:53,  1.11it/s]

 93%|██████████████████████████████████████████▋   | 3355/3614 [54:26<03:57,  1.09it/s]

 93%|██████████████████████████████████████████▋   | 3356/3614 [54:27<03:51,  1.11it/s]

 93%|██████████████████████████████████████████▋   | 3357/3614 [54:28<03:40,  1.17it/s]

 93%|██████████████████████████████████████████▋   | 3358/3614 [54:29<04:10,  1.02it/s]

 93%|██████████████████████████████████████████▊   | 3359/3614 [54:30<04:00,  1.06it/s]

 93%|██████████████████████████████████████████▊   | 3360/3614 [54:31<04:21,  1.03s/it]

 93%|██████████████████████████████████████████▊   | 3361/3614 [54:33<04:49,  1.14s/it]

 93%|██████████████████████████████████████████▊   | 3362/3614 [54:33<04:22,  1.04s/it]

 93%|██████████████████████████████████████████▊   | 3363/3614 [54:34<04:08,  1.01it/s]

 93%|██████████████████████████████████████████▊   | 3364/3614 [54:35<04:09,  1.00it/s]

 93%|████████████████

 95%|███████████████████████████████████████████▊  | 3440/3614 [55:50<02:38,  1.10it/s]

 95%|███████████████████████████████████████████▊  | 3441/3614 [55:51<02:38,  1.09it/s]

 95%|███████████████████████████████████████████▊  | 3442/3614 [55:52<02:30,  1.15it/s]

 95%|███████████████████████████████████████████▊  | 3443/3614 [55:52<02:25,  1.18it/s]

 95%|███████████████████████████████████████████▊  | 3444/3614 [55:53<02:22,  1.20it/s]

 95%|███████████████████████████████████████████▊  | 3445/3614 [55:54<02:40,  1.05it/s]

 95%|███████████████████████████████████████████▊  | 3446/3614 [55:55<02:41,  1.04it/s]

 95%|███████████████████████████████████████████▊  | 3447/3614 [55:56<02:35,  1.07it/s]

 95%|███████████████████████████████████████████▉  | 3448/3614 [55:57<02:25,  1.14it/s]

 95%|███████████████████████████████████████████▉  | 3449/3614 [55:58<02:19,  1.18it/s]

 95%|███████████████████████████████████████████▉  | 3450/3614 [55:59<02:15,  1.21it/s]

 95%|████████████████

 98%|████████████████████████████████████████████▉ | 3526/3614 [57:17<01:21,  1.08it/s]

 98%|████████████████████████████████████████████▉ | 3527/3614 [57:18<01:22,  1.05it/s]

 98%|████████████████████████████████████████████▉ | 3528/3614 [57:20<01:41,  1.18s/it]

 98%|████████████████████████████████████████████▉ | 3529/3614 [57:20<01:30,  1.06s/it]

 98%|████████████████████████████████████████████▉ | 3530/3614 [57:21<01:24,  1.00s/it]

 98%|████████████████████████████████████████████▉ | 3531/3614 [57:22<01:17,  1.07it/s]

 98%|████████████████████████████████████████████▉ | 3532/3614 [57:23<01:13,  1.12it/s]

 98%|████████████████████████████████████████████▉ | 3533/3614 [57:24<01:25,  1.05s/it]

 98%|████████████████████████████████████████████▉ | 3534/3614 [57:25<01:21,  1.02s/it]

 98%|████████████████████████████████████████████▉ | 3535/3614 [57:26<01:18,  1.00it/s]

 98%|█████████████████████████████████████████████ | 3536/3614 [57:27<01:16,  1.02it/s]

 98%|████████████████

100%|█████████████████████████████████████████████▉| 3612/3614 [58:36<00:01,  1.04it/s]

100%|█████████████████████████████████████████████▉| 3613/3614 [58:37<00:00,  1.04it/s]

100%|██████████████████████████████████████████████| 3614/3614 [58:38<00:00,  1.03it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3614 entries, 0 to 3613
Data columns (total 642 columns):
accuracy_group            3614 non-null int64
installation_id           3614 non-null object
game_time_p50             3614 non-null int32
game_time_max             3614 non-null int32
event_count_p50           3614 non-null int32
event_count_max           3614 non-null int32
type_activity             3614 non-null int32
type_assessment           3614 non-null int32
type_clip                 3614 non-null int32
type_game                 3614 non-null int32
world_crystalcaves        3614 non-null int32
world_magmapeak           3614 non-null int32
world_treetopcity         3614 non-null int32
world_none                3614 non-null int32
title_12_monkeys          3614 non-null int32
title_air_show            3614 non-null int32
title_all_star_sorting    3614 non-null int32
title_balancing_act       3614 non-null int32
title_bird_measurer       3614 non-null int32
title_bottle_fill

In [34]:
train.head(10)

,accuracy_group,installation_id,game_time_p50,game_time_max,event_count_p50,event_count_max,type_activity,type_assessment,type_clip,type_game,world_crystalcaves,world_magmapeak,world_treetopcity,world_none,title_12_monkeys,title_air_show,title_all_star_sorting,title_balancing_act,title_bird_measurer,title_bottle_filler,title_bubble_bath,title_bug_measurer,title_cart_balancer,title_cauldron_filler,title_chest_sorter,title_chicken_balancer,title_chow_time,title_costume_box,title_crystal_caves_1,title_crystal_caves_2,title_crystal_caves_3,title_crystals_rule,title_dino_dive,title_dino_drink,title_egg_dropper,title_fireworks,title_flower_waterer,title_happy_camel,title_heavy,title_honey_cake,title_leaf_leader,title_lifting,title_magma_peak_1,title_magma_peak_2,title_mushroom_sorter,title_ordering_spheres,title_pan_balance,title_pirate_tale,title_rulers,title_sandcastle,title_scrub,title_slop,title_treasure_map,title_treetop_city_1,title_treetop_city_2,title_treetop_city_3,title_watering_hole,title_welcome,event_2000,event_2000_activity,event_2000_assessment,event_2000_clip,event_2000_game,event_2010,event_2010_activity,event_2010_assessment,event_2010_clip,event_2010_game,event_2020,event_2020_activity,event_2020_assessment,event_2020_clip,event_2020_game,event_2025,event_2025_activity,event_2025_assessment,event_2025_clip,event_2025_game,event_2030,event_2030_activity,event_2030_assessment,event_2030_clip,event_2030_game,event_2035,event_2035_activity,event_2035_assessment,event_2035_clip,event_2035_game,event_2040,event_2040_activity,event_2040_assessment,event_2040_clip,event_2040_game,event_2050,event_2050_activity,event_2050_assessment,event_2050_clip,event_2050_game,event_2060,event_2060_activity,event_2060_assessment,event_2060_clip,event_2060_game,event_2070,event_2070_activity,event_2070_assessment,event_2070_clip,event_2070_game,event_2075,event_2075_activity,event_2075_assessment,event_2075_clip,event_2075_game,event_2080,event_2080_activity,event_2080_assessment,event_2080_clip,event_2080_game,event_2081,event_2081_activity,event_2081_assessment,event_2081_clip,event_2081_game,event_2083,event_2083_activity,event_2083_assessment,event_2083_clip,event_2083_game,event_3010,event_3010_activity,event_3010_assessment,event_3010_clip,event_3010_game,event_3020,event_3020_activity,event_3020_assessment,event_3020_clip,event_3020_game,event_3021,event_3021_activity,event_3021_assessment,event_3021_clip,event_3021_game,event_3110,event_3110_activity,event_3110_assessment,event_3110_clip,event_3110_game,event_3120,event_3120_activity,event_3120_assessment,event_3120_clip,event_3120_game,event_3121,event_3121_activity,event_3121_assessment,event_3121_clip,event_3121_game,event_4010,event_4010_activity,event_4010_assessment,event_4010_clip,event_4010_game,event_4020,event_4020_activity,event_4020_assessment,event_4020_clip,event_4020_game,event_4021,event_4021_activity,event_4021_assessment,event_4021_clip,event_4021_game,event_4022,event_4022_activity,event_4022_assessment,event_4022_clip,event_4022_game,event_4025,event_4025_activity,event_4025_assessment,event_4025_clip,event_4025_game,event_4030,event_4030_activity,event_4030_assessment,event_4030_clip,event_4030_game,event_4031,event_4031_activity,event_4031_assessment,event_4031_clip,event_4031_game,event_4035,event_4035_activity,event_4035_assessment,event_4035_clip,event_4035_game,event_4040,event_4040_activity,event_4040_assessment,event_4040_clip,event_4040_game,event_4045,event_4045_activity,event_4045_assessment,event_4045_clip,event_4045_game,event_4050,event_4050_activity,event_4050_assessment,event_4050_clip,event_4050_game,event_4070,event_4070_activity,event_4070_assessment,event_4070_clip,event_4070_game,event_4080,event_4080_activity,event_4080_assessment,event_4080_clip,event_4080_game,event_4090,event_4090_activity,event_4090_assessment,event_4090_clip,event_4090_game,event_4095,event_4095_activity,event_4095_assessment,event_4095_clip,event_4095_game,event_41

In [35]:
test = _preprocess(test_raw, EVENT_CODES, EVENT_IDS)
test.info(max_cols=999)



  0%|                                                         | 0/1000 [00:00<?, ?it/s]

  0%|                                                 | 1/1000 [00:00<07:00,  2.38it/s]

  0%|                                                 | 2/1000 [00:01<08:07,  2.05it/s]

  0%|▏                                                | 3/1000 [00:01<07:26,  2.23it/s]

  0%|▏                                                | 4/1000 [00:01<07:00,  2.37it/s]

  0%|▏                                                | 5/1000 [00:02<07:00,  2.37it/s]

  1%|▎                                                | 6/1000 [00:02<06:58,  2.37it/s]

  1%|▎                                                | 7/1000 [00:03<06:55,  2.39it/s]

  1%|▍                                                | 8/1000 [00:03<07:31,  2.20it/s]

  1%|▍                                                | 9/1000 [00:04<07:49,  2.11it/s]

  1%|▍                                               | 10/1000 [00:04<07:08,  2.31it/s]

  1%|▌             

  9%|████▏                                           | 86/1000 [00:40<06:11,  2.46it/s]

  9%|████▏                                           | 87/1000 [00:40<05:56,  2.56it/s]

  9%|████▏                                           | 88/1000 [00:41<06:56,  2.19it/s]

  9%|████▎                                           | 89/1000 [00:42<08:22,  1.81it/s]

  9%|████▎                                           | 90/1000 [00:42<07:59,  1.90it/s]

  9%|████▎                                           | 91/1000 [00:42<07:08,  2.12it/s]

  9%|████▍                                           | 92/1000 [00:43<06:56,  2.18it/s]

  9%|████▍                                           | 93/1000 [00:43<06:31,  2.32it/s]

  9%|████▌                                           | 94/1000 [00:44<06:01,  2.51it/s]

 10%|████▌                                           | 95/1000 [00:44<06:05,  2.48it/s]

 10%|████▌                                           | 96/1000 [00:44<06:20,  2.37it/s]

 10%|████▋           

 17%|████████                                       | 172/1000 [01:17<05:02,  2.74it/s]

 17%|████████▏                                      | 173/1000 [01:17<04:56,  2.79it/s]

 17%|████████▏                                      | 174/1000 [01:18<05:47,  2.38it/s]

 18%|████████▏                                      | 175/1000 [01:19<07:50,  1.75it/s]

 18%|████████▎                                      | 176/1000 [01:19<07:04,  1.94it/s]

 18%|████████▎                                      | 177/1000 [01:20<06:50,  2.01it/s]

 18%|████████▎                                      | 178/1000 [01:20<06:07,  2.23it/s]

 18%|████████▍                                      | 179/1000 [01:20<05:55,  2.31it/s]

 18%|████████▍                                      | 180/1000 [01:21<05:48,  2.35it/s]

 18%|████████▌                                      | 181/1000 [01:21<05:26,  2.51it/s]

 18%|████████▌                                      | 182/1000 [01:22<06:49,  2.00it/s]

 18%|████████▌       

 26%|████████████▏                                  | 258/1000 [01:57<04:56,  2.51it/s]

 26%|████████████▏                                  | 259/1000 [01:57<05:17,  2.33it/s]

 26%|████████████▏                                  | 260/1000 [01:58<05:48,  2.12it/s]

 26%|████████████▎                                  | 261/1000 [01:58<05:45,  2.14it/s]

 26%|████████████▎                                  | 262/1000 [01:58<05:16,  2.33it/s]

 26%|████████████▎                                  | 263/1000 [01:59<04:55,  2.49it/s]

 26%|████████████▍                                  | 264/1000 [01:59<04:44,  2.59it/s]

 26%|████████████▍                                  | 265/1000 [02:00<05:59,  2.04it/s]

 27%|████████████▌                                  | 266/1000 [02:00<05:20,  2.29it/s]

 27%|████████████▌                                  | 267/1000 [02:00<04:56,  2.47it/s]

 27%|████████████▌                                  | 268/1000 [02:01<04:41,  2.60it/s]

 27%|████████████▋   

 34%|████████████████▏                              | 344/1000 [02:35<04:04,  2.68it/s]

 34%|████████████████▏                              | 345/1000 [02:35<04:00,  2.72it/s]

 35%|████████████████▎                              | 346/1000 [02:35<03:56,  2.77it/s]

 35%|████████████████▎                              | 347/1000 [02:36<03:48,  2.86it/s]

 35%|████████████████▎                              | 348/1000 [02:36<04:46,  2.28it/s]

 35%|████████████████▍                              | 349/1000 [02:37<05:14,  2.07it/s]

 35%|████████████████▍                              | 350/1000 [02:37<05:35,  1.94it/s]

 35%|████████████████▍                              | 351/1000 [02:38<04:59,  2.16it/s]

 35%|████████████████▌                              | 352/1000 [02:38<04:39,  2.32it/s]

 35%|████████████████▌                              | 353/1000 [02:39<04:18,  2.50it/s]

 35%|████████████████▋                              | 354/1000 [02:39<04:05,  2.63it/s]

 36%|████████████████

 43%|████████████████████▏                          | 430/1000 [03:09<03:41,  2.58it/s]

 43%|████████████████████▎                          | 431/1000 [03:10<03:48,  2.49it/s]

 43%|████████████████████▎                          | 432/1000 [03:10<03:55,  2.42it/s]

 43%|████████████████████▎                          | 433/1000 [03:10<03:39,  2.59it/s]

 43%|████████████████████▍                          | 434/1000 [03:11<04:48,  1.96it/s]

 44%|████████████████████▍                          | 435/1000 [03:12<04:19,  2.18it/s]

 44%|████████████████████▍                          | 436/1000 [03:12<04:04,  2.30it/s]

 44%|████████████████████▌                          | 437/1000 [03:12<04:15,  2.20it/s]

 44%|████████████████████▌                          | 438/1000 [03:13<04:03,  2.31it/s]

 44%|████████████████████▋                          | 439/1000 [03:13<04:08,  2.26it/s]

 44%|████████████████████▋                          | 440/1000 [03:14<03:52,  2.41it/s]

 44%|████████████████

 52%|████████████████████████▎                      | 516/1000 [03:52<03:57,  2.03it/s]

 52%|████████████████████████▎                      | 517/1000 [03:53<03:38,  2.21it/s]

 52%|████████████████████████▎                      | 518/1000 [03:53<03:34,  2.25it/s]

 52%|████████████████████████▍                      | 519/1000 [03:53<03:18,  2.43it/s]

 52%|████████████████████████▍                      | 520/1000 [03:54<03:18,  2.42it/s]

 52%|████████████████████████▍                      | 521/1000 [03:54<03:15,  2.45it/s]

 52%|████████████████████████▌                      | 522/1000 [03:55<03:05,  2.57it/s]

 52%|████████████████████████▌                      | 523/1000 [03:55<02:59,  2.66it/s]

 52%|████████████████████████▋                      | 524/1000 [03:55<02:56,  2.69it/s]

 52%|████████████████████████▋                      | 525/1000 [03:56<02:56,  2.70it/s]

 53%|████████████████████████▋                      | 526/1000 [03:56<03:21,  2.35it/s]

 53%|████████████████

 60%|████████████████████████████▎                  | 602/1000 [04:32<03:08,  2.11it/s]

 60%|████████████████████████████▎                  | 603/1000 [04:32<02:58,  2.23it/s]

 60%|████████████████████████████▍                  | 604/1000 [04:33<03:23,  1.95it/s]

 60%|████████████████████████████▍                  | 605/1000 [04:33<02:59,  2.20it/s]

 61%|████████████████████████████▍                  | 606/1000 [04:34<03:16,  2.01it/s]

 61%|████████████████████████████▌                  | 607/1000 [04:34<03:18,  1.97it/s]

 61%|████████████████████████████▌                  | 608/1000 [04:35<03:16,  2.00it/s]

KeyboardInterrupt: 

In [ ]:
test.head(20)

In [ ]:
train.to_parquet('train.parquet')
test.to_parquet('test.parquet')
_log(os.listdir("."))

# Train Model

In [ ]:
train = pd.read_parquet('train.parquet')
test = pd.read_parquet('test.parquet')

In [ ]:
PREDICTORS = set(test.columns.values) - {'installation_id'}

In [ ]:
def _rmse(y, y_pred):
    return sqrt(mean_squared_error(y, y_pred))


SCORING = make_scorer(_rmse, greater_is_better = False)

In [ ]:
%%time
y_train = train[TARGET].astype(int)
x_train = train[PREDICTORS]
model = lgb.LGBMRegressor(n_estimators=ESTIMATORS, reg_alpha=1)
pipe = Pipeline([('model', model)])
param_grid = {
    'model__learning_rate': [0.01],
    'model__num_leaves': [80],
    'model__min_child_samples': [200],
    'model__colsample_bytree': [0.5]
}
cv = GridSearchCV(pipe, cv=FOLDS, param_grid=param_grid, scoring=SCORING)
#cv.fit(x_train, y_train, model__early_stopping_rounds=200, model__verbose=500)
cv.fit(x_train, y_train)
#assert cv.best_estimator_['model'].n_classes_ == 4
_log(f'best_params_={cv.best_params_}\nbest_score_={cv.best_score_:.5f}')

In [ ]:
# plot_metric only works with early stopping rounds
#lgb.plot_metric(cv.best_estimator_['model'])

In [ ]:
lgb.plot_importance(cv.best_estimator_['model'], max_num_features=100, figsize=(10, 30))

# Predict out of fold

In [ ]:
oof = train[['installation_id']].copy()
oof[TARGET] = cv.predict(x_train)
assert oof[TARGET].min() > -0.5
assert oof[TARGET].max() < 3.5
oof[TARGET] = np.round(oof[TARGET]).astype(int)
oof.head()

In [ ]:
score = cohen_kappa_score(oof[TARGET], y_train, weights='quadratic')
_log(f'oof score={score:.5f}')

# Predict on Test set

In [ ]:
x_test = test[PREDICTORS]
sub = test[['installation_id']].copy()
sub[TARGET] = cv.predict(x_test)
assert sub[TARGET].min() > -0.5
assert sub[TARGET].max() < 3.5
sub[TARGET] = np.round(sub[TARGET]).astype(int)
sub.head()

In [ ]:
plt.subplot(1, 3, 1)
plt.title('test predict')
sub[TARGET].plot(kind='hist')
plt.subplot(1, 3, 2)
plt.title('oof predict')
oof[TARGET].plot(kind='hist')
plt.subplot(1, 3, 3)
plt.title('oof truth')
tmp = train[TARGET].copy()
tmp = tmp.astype(int)
tmp.plot(kind='hist')
plt.tight_layout()

In [ ]:
sub.to_csv('submission.csv', index=False)
_log(os.listdir("."))